## 1. Import and Install Dependencies

In [5]:
# Install the following packages on your pc if doing so here doesn't work
# !pip install tensorflow opencv-python mediapipe sklearn matplotlib

  Using cached sklearn-0.0.post9-py3-none-any.whl


In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot
import time
import mediapipe

## 2. Keypoints using MP Holistic

In [2]:
mp_holistic = mediapipe.solutions.holistic # Holistic model # detects points
mp_drawing = mediapipe.solutions.drawing_utils # Drawing utilities # draws out detection

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # converts color space from BGR to RGB which saves some memory
    image.flags.writeable = False                  # marks image as unwriteable
    results = model.process(image)                 # Makes prediction
    image.flags.writeable = True                   # converts back
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # converts back
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [5]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                                 mp_drawing.DrawingSpec(color=(68,64,71), thickness=1, circle_radius=1), # joint spec
                                 mp_drawing.DrawingSpec(color=(119,155,0), thickness=1, circle_radius=1) # line spec 
                             )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), # joint spec
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2) # line spec
                             )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), # joint spec
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2) # line spec
                             )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), # joint spec
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) # line spec
                             )

In [6]:
# cap = cv2.VideoCapture(0) # access webcam

# # Set mediapipe model
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     while cap.isOpened(): # while accessing webcam
#         # read feed (the current frame)
#         ret, frame = cap.read() 
        
#         # Make detections
#         image, results = mediapipe_detection(frame, holistic) # holistic: model

#         # Draw landmarks
#         draw_styled_landmarks(image, results)
        
#         # show to screen (frame name, actual frame)
#         cv2.imshow('OpenCV Feed', image) 
        
#         # condition to close gracefully WHEN:
#         #     waited for 0.01 sec for a keypress & keypress is 'q', OR
#         #     the [X] button on the window is clicked
#         if (cv2.waitKey(10) & 0xFF == ord('q')) or (cv2.getWindowProperty('OpenCV Feed', cv2.WND_PROP_VISIBLE) < 1): 
#             break
        
# cap.release()
# cv2.destroyAllWindows()

## 3. Extract Keypoint Values

In [7]:
# pose = []
# for res in results.pose_landmarks.landmark:
#     test = np.array([res.x, res.y, res.z, res.visibility])
#     pose.append(test)

In [8]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    leftHand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rightHand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, leftHand, rightHand])

## 4. Setup Folders for Collection

In [9]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data')

# Action that we try to detect
# actions = np.array(['hello', 'thanks', 'iloveyou', 'I', 'you', 'deaf', 'hearing', 'what_question', 'what_relative_clause'])
actions = np.array(['hello', 'thanks', 'iloveyou'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30


In [10]:
# Create folders for each sign:
# hello
## 0
## 1
## ...
## 29
# thanks ...
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

## 5. Collect Keypoint Values for Training and Testing

In [11]:
# cap = cv2.VideoCapture(0) # access webcam

# # Set mediapipe model
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     breakout = False
#     # Loop through actions
#     for action in actions:
#         if breakout:
#             break
#         # Loop through sequences aka videos
#         for sequence in range(no_sequences):
#             if breakout:
#                 break
#             # Loop through video length aka sequence length
#             for frame_num in range(sequence_length):
            
#                 # read feed (the current frame)
#                 ret, frame = cap.read() 
                
#                 # Make detections
#                 image, results = mediapipe_detection(frame, holistic) # holistic: model
        
#                 # Draw landmarks
#                 draw_styled_landmarks(image, results)
    
#                 # Apply wait logic
#                 if frame_num == 0:
#                     cv2.putText(image, 'STARTING COLLECTION', (120, 200),
#                                     cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4, cv2.LINE_AA)
#                     cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12),
#                                     cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
#                     cv2.waitKey(500)
#                 else:
#                     cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12),
#                                     cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
#                 # Export keypoints
#                 keypoints = extract_keypoints(results)
#                 npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
#                 np.save(npy_path, keypoints)
                                
#                 # show to screen (frame name, actual frame)
#                 cv2.imshow('OpenCV Feed', image) 
                
#                 # condition to close gracefully WHEN:
#                 #     waited for 0.01 sec for a keypress & keypress is 'q', OR
#                 #     the [X] button on the window is clicked
#                 if (cv2.waitKey(10) & 0xFF == ord('q')) or (cv2.getWindowProperty('OpenCV Feed', cv2.WND_PROP_VISIBLE) < 1): 
#                     breakout = True
#                     break
            
#     cap.release()
#     cv2.destroyAllWindows()

## 6. Preprocess Data and Create Labels and Features

In [12]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [13]:
label_map = {label:num for num, label in enumerate(actions)}

In [14]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [15]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [16]:
x = np.array(sequences)
y = to_categorical(labels).astype(int)

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05)

## 7. Build and Train LSTM Neural Network

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [19]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir) # web app to monitor training

In [20]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662))) # 30 frames, 1662 keypoints
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [21]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [22]:
model.fit(x_train, y_train, epochs=200, callbacks=[tb_callback])

Epoch 1/200
3/3 [==============================] - 20s 208ms/step - loss: 5.8814 - categorical_accuracy: 0.3412
Epoch 2/200
3/3 [==============================] - 1s 207ms/step - loss: 1.8623 - categorical_accuracy: 0.3412
Epoch 3/200
3/3 [==============================] - 1s 195ms/step - loss: 1.4988 - categorical_accuracy: 0.3529
Epoch 4/200
3/3 [==============================] - 1s 172ms/step - loss: 1.0801 - categorical_accuracy: 0.4118
Epoch 5/200
3/3 [==============================] - 1s 192ms/step - loss: 0.8147 - categorical_accuracy: 0.6588
Epoch 6/200
3/3 [==============================] - 1s 182ms/step - loss: 0.6165 - categorical_accuracy: 0.8824
Epoch 7/200
3/3 [==============================] - 1s 199ms/step - loss: 0.4090 - categorical_accuracy: 0.8471
Epoch 8/200
3/3 [==============================] - 1s 200ms/step - loss: 0.6890 - categorical_accuracy: 0.6824
Epoch 9/200
3/3 [==============================] - 1s 180ms/step - loss: 0.6820 - categorical_accuracy: 0.6353


In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596675 (2.28 MB)
Trainable params: 596675 

## 8.Make Predictions

In [24]:
res = model.predict(x_test)

1/1 [==============================] - 2s 2s/step


In [25]:
actions[np.argmax(res[2])]

'iloveyou'

In [26]:
actions[np.argmax(y_test[2])]

'iloveyou'

In [27]:
# res[np.argmax(res)]

## 9. Save Weights

In [28]:
# model.save('action.keras') # save model

In [29]:
#model.load_weights('action.keras') # reload model after initializing and compiling (7)

## Evaluation using Confusion Matrix and Accuracy

In [30]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [31]:
yhat = model.predict(x_test)

1/1 [==============================] - 0s 106ms/step


In [32]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [33]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[3, 0],
        [0, 2]],

       [[2, 0],
        [0, 3]]], dtype=int64)

In [34]:
accuracy_score(ytrue, yhat)

1.0

## 11. Test in Real Time

In [39]:
# 1. Detection Variables
sequence = []
sentence = []
threshold = 0.3

cap = cv2.VideoCapture(0) # access webcam

# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened(): # while accessing webcam
        # read feed (the current frame)
        # timer = 0
        ret, frame = cap.read() 
        
        # Make detections
        image, results = mediapipe_detection(frame, holistic) # holistic: model

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Prediction logic
        keypoints = extract_keypoints(results)
        #sequence.append(keypoints)
        sequence = sequence[-30:]

        #res = []
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
        
            # visualization logic
            if res[np.argmax(res)] > threshold:
                if len(sentence) > 0:
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
            else:
                sentence.append(actions[np.argmax(res)])
                    # timer += 1
    
        if len(sentence) > 5:
            sentence = sentence[-5:]

        cv2.rectangle(image, (0,0), (640,40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        # show to screen (frame name, actual frame)
        cv2.imshow('OpenCV Feed', image) 
        
        # condition to close gracefully WHEN:
        #     waited for 0.01 sec for a keypress & keypress is 'q', OR
        #     the [X] button on the window is clicked
        if (cv2.waitKey(10) & 0xFF == ord('q')) or (cv2.getWindowProperty('OpenCV Feed', cv2.WND_PROP_VISIBLE) < 1): 
            break
        
cap.release()
cv2.destroyAllWindows()

In [36]:
model.predict??

Signature:
model.predict(
    x,
    batch_size=None,
    verbose='auto',
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)
Source:   
    @traceback_utils.filter_traceback
    def predict(
        self,
        x,
        batch_size=None,
        verbose="auto",
        steps=None,
        callbacks=None,
        max_queue_size=10,
        workers=1,
        use_multiprocessing=False,
    ):
        """Generates output predictions for the input samples.

        Computation is done in batches. This method is designed for batch
        processing of large numbers of inputs. It is not intended for use inside
        of loops that iterate over your data and process small numbers of inputs
        at a time.

        For small numbers of inputs that fit in one batch,
        directly use `__call__()` for faster execution, e.g.,
        `model(x)`, or `model(x, training=False)` if you have layers such as
        `tf.keras.layers.Batc